In [1]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
model_name = 'LogisticRegression'

mlq.start_quest(model_name, table_dest="../../",log_defs=True)

# <font color="aqua">Logistic Regression</font> Model

### Read the data

In [2]:
x_data_d, y_data_d = mlq.l(read_data)(split='all',kind='Numerical')
opt_params = load_hyperparameters(model_name)

### Initiate model

In [3]:
clf = mlq.l(LogisticRegression)(**opt_params)
# clf = mlq.l(LogisticRegression)(multi_class='multinomial', C=1400, solver='newton-cg')

### Evaluate Model Bias

In [4]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
_, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.984     0.995     0.990       190
           1      0.966     0.980     0.973       201
           2      0.993     0.985     0.989       406
           3      1.000     0.997     0.999       680

    accuracy                          0.991      1477
   macro avg      0.986     0.989     0.987      1477
weighted avg      0.991     0.991     0.991      1477



### Evaluate Model Generalization

In [5]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1,loo=False)

val_wf1 = kfold.get(f'10-Repeated 10-fold')[0] 
# repeated_10fold_report= kfold.get(f'10-Repeated 10-fold')[1]
# print(f'10-Repeated 10-fold Report:\n {repeated_10fold_report}')

# loo_wf1=loo.get('loo_wf1')
# loo_report=loo.get('loo_report')
# print(f'Leave-One-Out Report:\n {loo_report}')

10-Repeated 10-fold
0.9831


#### Save Model

In [6]:
save_model(model_name, clf)

### Tracking

In [7]:
mlq.log_metrics(train_wf1,val_wf1)
mlq.end_quest()
mlq.show_logs("LogisticRegression", table_dest="../../", last_k=6)